# US Overdose Insights — Python → Snowflake → Tableau

This notebook cleans and models the **Accidental Drug Related Deaths (2012–2024)** dataset and prepares it for a simple analytics pipeline:

**Python (pandas) → Snowflake (staging table) → Tableau dashboard**

## What an interviewer should look for
- Clear, reproducible data cleaning (column standardization, geo parsing, missing-value strategy)
- Basic quality checks to prevent silent data issues
- Time-series aggregation and lightweight forecasting (Prophet)
- Production-minded exports (idempotent load to Snowflake; optional, gated by environment variables)

> **Note:** Snowflake credentials are **not** hard-coded. Set environment variables (see _Configuration_ below).


## Configuration

### Data
Place the raw CSV in `data/` (recommended for GitHub):

- `data/Accidental_Drug_Related_Deaths_2012-2024.csv`

### Python environment
Recommended:
```bash
pip install -r requirements.txt
```

Minimum packages used here:
- pandas, numpy
- matplotlib
- prophet (optional; only needed for forecasting)
- snowflake-connector-python[pandas] (optional; only needed to load to Snowflake)

### Snowflake (optional)
Set these environment variables if you want to run the Snowflake load:
- `SNOWFLAKE_USER`
- `SNOWFLAKE_PASSWORD`
- `SNOWFLAKE_ACCOUNT`
- `SNOWFLAKE_WAREHOUSE`
- `SNOWFLAKE_DATABASE`
- `SNOWFLAKE_SCHEMA`

If they are not set, the Snowflake section will be skipped safely.


In [ ]:
# ---- Parameters (edit here) ----
from pathlib import Path
import os

DATA_PATH = Path("data/Accidental_Drug_Related_Deaths_2012-2024.csv")

# Forecast horizon (months)
FORECAST_PERIODS = 12

# Export toggles
EXPORT_CLEAN_CSV = True
EXPORT_TO_SNOWFLAKE = True  # will auto-skip if env vars are missing

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## 1) Helpers

In [ ]:
import re

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    '''Return a copy with snake_case column names and trimmed whitespace.'''
    out = df.copy()
    out.columns = (
        out.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^\w\s]", "", regex=True)
        .str.replace(r"\s+", "_", regex=True)
    )
    return out

_geo_re = re.compile(r"\((.*?),\s*(.*?)\)")

def extract_lat_long(series: pd.Series) -> pd.DataFrame:
    '''Extract lat/long from strings like '(41.7, -72.6)'. Returns two float columns.'''
    extracted = series.astype("string").str.extract(_geo_re)
    extracted.columns = ["latitude", "longitude"]
    return extracted.astype("float64")

_state_re = re.compile(r"(?:^|,)\s*([A-Z]{2})\s*$")

def extract_state_from_geo(series: pd.Series) -> pd.Series:
    '''Extract state code from geo strings like 'Hartford, CT'.'''
    return series.astype("string").str.extract(_state_re)[0]

def clean_title(series: pd.Series) -> pd.Series:
    '''Strip, title-case, and replace empty with NaN.'''
    s = series.astype("string").str.strip()
    s = s.replace({"": pd.NA})
    return s.str.title()

def clean_upper(series: pd.Series) -> pd.Series:
    s = series.astype("string").str.strip()
    s = s.replace({"": pd.NA})
    return s.str.upper()


## 2) Load raw data

In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Missing {DATA_PATH}. Put the CSV under ./data/ (see the Configuration section)."
    )

df_raw = pd.read_csv(DATA_PATH)
df_raw.shape


### Quick sanity check

In [ ]:
display(df_raw.head(3))
display(df_raw.tail(3))

# Missingness overview (top 15)
missing = (df_raw.isna().mean().sort_values(ascending=False).head(15) * 100).round(1)
missing


## 3) Clean & standardize

In [ ]:
df = standardize_columns(df_raw)

# Example: many city/county fields are inconsistent in casing; normalize key text columns
text_title_cols = [
    "residence_city", "residence_county", "injury_city", "injury_county", "death_city", "death_county"
]
for col in [c for c in text_title_cols if c in df.columns]:
    df[col] = clean_title(df[col])

# Normalize state columns to uppercase 2-letter codes when possible
state_cols = ["residence_state", "injury_state", "death_state"]
for col in [c for c in state_cols if c in df.columns]:
    df[col] = clean_upper(df[col])

# Parse geo fields to numeric lat/long (if present)
geo_map = {
    "residencecitygeo": ("rc_latitude", "rc_longitude"),
    "injurycitygeo": ("ic_latitude", "ic_longitude"),
    "deathcitygeo": ("dc_latitude", "dc_longitude"),
}
for geo_col, (lat_col, lon_col) in geo_map.items():
    if geo_col in df.columns:
        coords = extract_lat_long(df[geo_col])
        df[lat_col] = coords["latitude"]
        df[lon_col] = coords["longitude"]

# Fill missing state using geo strings (most reliable among available fields)
if "residence_state" in df.columns and "deathcitygeo" in df.columns:
    df["residence_state"] = df["residence_state"].fillna(extract_state_from_geo(df["deathcitygeo"]))

# Common 'Unknown' fills for location dimensions
for col in ["residence_city", "residence_county", "injury_city", "injury_county", "death_city", "death_county"]:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

df.shape


### Dates & key fields

In [ ]:
# Parse date and derive year/month for analysis
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

# Basic integrity checks
assert df["date"].notna().mean() > 0.95, "Too many missing/invalid dates after parsing."
df[["date","year","month"]].head()


### Drug indicator columns

In [ ]:
# Many datasets encode drug presence as Y/N or 0/1. Standardize to 0/1 ints when present.
drug_columns = [c for c in df.columns if c.startswith("heroin") or c.startswith("cocaine") or c.startswith("fentanyl") or c.startswith("benzodiazepine") or c.startswith("alcohol")]
# If your dataset uses different names, extend this list.

def normalize_yes_no_to_int(s: pd.Series) -> pd.Series:
    s2 = s.astype("string").str.strip().str.upper()
    return s2.map({"Y": 1, "YES": 1, "TRUE": 1, "1": 1, "N": 0, "NO": 0, "FALSE": 0, "0": 0})

for col in drug_columns:
    df[col] = normalize_yes_no_to_int(df[col]).astype("Int64")

drug_columns[:10], len(drug_columns)


## 4) Analytics tables (for Tableau)

In [ ]:
# Deaths per month
death_per_month = (
    df.groupby("month", dropna=False)
      .size()
      .reset_index(name="death_count")
      .sort_values("month")
)

display(death_per_month.tail(6))

plt.figure()
plt.plot(death_per_month["month"], death_per_month["death_count"])
plt.title("Accidental drug-related deaths per month")
plt.xlabel("Month")
plt.ylabel("Deaths")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Example: deaths by race (if present)
if "race" in df.columns:
    deaths_by_race = df.groupby("race").size().reset_index(name="death_count").sort_values("death_count", ascending=False)
    display(deaths_by_race.head(10))
else:
    deaths_by_race = None


## 5) Forecasting (Prophet, optional)

In [ ]:
# Prophet is optional; if unavailable, we skip forecasting.
try:
    from prophet import Prophet  # type: ignore
    has_prophet = True
except Exception as e:
    has_prophet = False
    print("Prophet not installed or failed to import. Skipping forecasting. Details:", e)

forecasted_monthly_death = None

if has_prophet:
    df_prophet = death_per_month.rename(columns={"month": "ds", "death_count": "y"}).dropna()
    model = Prophet()
    model.fit(df_prophet)

    future = model.make_future_dataframe(periods=FORECAST_PERIODS, freq="M")
    fc = model.predict(future)

    forecasted_monthly_death = fc[["ds", "yhat", "yhat_lower", "yhat_upper"]].copy()
    forecasted_monthly_death.columns = ["month", "forecast", "lower_bound", "upper_bound"]
    display(forecasted_monthly_death.tail(6))

    # Plot
    plt.figure()
    plt.plot(death_per_month["month"], death_per_month["death_count"], label="actual")
    plt.plot(forecasted_monthly_death["month"], forecasted_monthly_death["forecast"], label="forecast")
    plt.title("Monthly deaths — actual vs forecast")
    plt.xlabel("Month")
    plt.ylabel("Deaths")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()


## 6) Exports

In [ ]:
# Clean dataset export (for Tableau local testing / sharing)
clean_csv_path = OUTPUT_DIR / "accidental_drug_related_deaths_clean.csv"
if EXPORT_CLEAN_CSV:
    df.to_csv(clean_csv_path, index=False)
    print("Wrote:", clean_csv_path)

# Optional: Snowflake load (skips if env vars missing)
def snowflake_env_ready() -> bool:
    required = [
        "SNOWFLAKE_USER", "SNOWFLAKE_PASSWORD", "SNOWFLAKE_ACCOUNT",
        "SNOWFLAKE_WAREHOUSE", "SNOWFLAKE_DATABASE", "SNOWFLAKE_SCHEMA",
    ]
    return all(os.getenv(k) for k in required)

if EXPORT_TO_SNOWFLAKE and snowflake_env_ready():
    import snowflake.connector
    from snowflake.connector.pandas_tools import write_pandas

    conn = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USER"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        warehouse=os.environ["SNOWFLAKE_WAREHOUSE"],
        database=os.environ["SNOWFLAKE_DATABASE"],
        schema=os.environ["SNOWFLAKE_SCHEMA"],
    )

    # Load cleaned table
    success, chunks, rows, _ = write_pandas(
        conn,
        df,
        table_name="ACCIDENTAL_DRUG_RELATED_DEATHS_CLEAN",
        auto_create_table=True,
    )
    print(f"Snowflake load (clean): success={success}, rows={rows}")

    # Load forecast table (if produced)
    if forecasted_monthly_death is not None:
        success_f, chunks_f, rows_f, _ = write_pandas(
            conn,
            forecasted_monthly_death,
            table_name="ACCIDENTAL_DRUG_RELATED_DEATHS_FORECAST",
            auto_create_table=True,
        )
        print(f"Snowflake load (forecast): success={success_f}, rows={rows_f}")

    conn.close()
else:
    if EXPORT_TO_SNOWFLAKE:
        print("Snowflake env vars not set — skipping Snowflake export.")


## Next steps (if you want to extend this)
- Add a `requirements.txt` and a small `Makefile` (or `uv` / `poetry`) so the notebook runs in one command.
- Add data tests (e.g., `great_expectations` or simple `pytest` checks) to guard schema + null rates.
- Add incremental loads (date-based) and a Snowflake task for scheduled refresh.
- Include a screenshot / link to your Tableau dashboard in the repository README.
